In [6]:
import librosa
import numpy as np
import pyworld

sound, sr = librosa.load('../voice/seika_voice/001.wav')
sound_double = sound.astype(np.float64)

# get info
f0, ap, sp =  pyworld.wav2world(sound_double, sr)
print(f0)
print(ap)
print(sp)

[  0.         237.24675    240.57799027 ... 177.74144098 177.12354187
 177.85948945]
[[1.02628879e-06 1.02716315e-06 1.02979459e-06 ... 1.00300298e-11
  9.32810758e-12 9.10188146e-12]
 [3.49510930e-06 3.51815227e-06 3.58280842e-06 ... 3.32550804e-10
  3.33930773e-10 3.34382431e-10]
 [1.68175772e-05 1.68616116e-05 1.69943391e-05 ... 9.30339811e-11
  8.58147868e-11 8.33529191e-11]
 ...
 [6.46139297e-04 6.46566391e-04 6.47624923e-04 ... 8.52724482e-10
  9.57860349e-10 9.95331763e-10]
 [3.73309012e-04 3.73236817e-04 3.73264052e-04 ... 7.52209196e-11
  6.50758183e-11 6.16191006e-11]
 [9.91760679e-05 9.91957283e-05 9.93368542e-05 ... 9.22824814e-11
  8.10316008e-11 7.74228782e-11]]
[[1.         1.         1.         ... 1.         1.         1.        ]
 [0.001      0.00104891 0.0011002  ... 0.99753274 0.99876561 1.        ]
 [0.001      0.00104866 0.00109969 ... 0.99924205 0.99962095 1.        ]
 ...
 [0.001      0.00104942 0.00110128 ... 0.99838981 0.99919458 1.        ]
 [0.001      0.001

* f0 = 基本周波数
* ap = 非周期性
* sp = スペクトログラム

In [8]:
import librosa
import wave
import numpy as np
import pyworld
import pyaudio
import soundfile as sf

# get sound info
sound, sr = librosa.load('../voice/seika_voice/001.wav')
sound_double = sound.astype(np.float64)
f0, sp, ap =  pyworld.wav2world(sound_double, sr)

# synthesize
make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
sf.write('aaa.wav', make_sound, sr)

## DTWでパラレルデータにしなきゃいけないっぽい

In [26]:
import librosa
import numpy as np
import pyworld
import soundfile as sf
from numpy.linalg import norm
from dtw import dtw

# get seika_sound info
sound, sr = librosa.load('../voice/seika_voice/001.wav')
seika_mfcc = librosa.feature.mfcc(y = sound, sr = sr)
#seika_sound_double = sound.astype(np.float64)
#f0, sp, ap =  pyworld.wav2world(seika_sound_double, seika_sr)

# get hiroshiba_sound info
sound, sr = librosa.load('../voice/hiroshiba_voice/001.wav')
hiro_mfcc = librosa.feature.mfcc(y = sound, sr = sr)
#hiro_sound_double = sound.astype(np.float64)
#f0, sp, ap =  pyworld.wav2world(hiro_sound_double, hiro_sr)

#DTW
dist, cost, acc_cost, path = dtw(seika_mfcc.T, hiro_mfcc.T, dist = lambda x, y: norm(x - y, ord = 1)) 
seika_conv_mfcc = seika_mfcc.T[path[0]].T
hiro_conv_mfcc = hiro_mfcc.T[path[1]].T
print(seika_conv_mfcc.shape)
print(hiro_conv_mfcc.shape)

# synthesize
 #make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
 #sf.write('aaa.wav', make_sound, sr)

(20, 318)
(20, 318)


## ISTFT(逆短時間フーリエ変換)をすると音声にできる？

In [19]:
import librosa
import numpy as np
import pyworld
import soundfile as sf
from numpy.linalg import norm
from dtw import dtw

# get wavefile info
def get_wave_info(file_name):
    sound, sr = librosa.load(file_name)
    mfcc = librosa.feature.mfcc(y = sound, sr = sr)
    stft = librosa.core.stft(sound)
    return sr, mfcc, stft

# do DTW
def DTW(mfcc_1, mfcc_2):
    dist, cost, acc_cost, path = dtw(mfcc_1.T, mfcc_2.T, dist = lambda x, y: norm(x - y, ord = 1))
    return path

# DTWed stft to sound
def stft_to_sound(stft, path):
    conv_stft = stft.T[path].T
    print(conv_stft.shape)
    conv_sound = librosa.core.istft(conv_stft)
    return conv_sound

# data to wave
def make_sound(sound, sr, file_name):
    sound_double = sound.astype(np.float64)
    f0, sp, ap = pyworld.wav2world(sound_double, sr)
    make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
    sf.write(file_name, make_sound, sr)
    
seika_sr,  seika_mfcc, seika_stft = get_wave_info('../voice/seika_silence/001.wav')
hiro_sr,  hiro_mfcc, hiro_stft = get_wave_info('../voice/hiro_silence/001.wav')

path = DTW(seika_mfcc, hiro_mfcc)

seika_conv_sound = stft_to_sound(seika_stft, path[0])
hiro_conv_sound = stft_to_sound(hiro_stft, path[1])

make_sound(seika_conv_sound, seika_sr, 'seika001.wav')
make_sound(hiro_conv_sound, hiro_sr, 'hiro001.wav')

(1025, 258)
(1025, 258)


* mfccとstftを対応づけたらできました！

## 流石にGANを作ろう

## その前にデータ群を作ろう

In [2]:
import librosa
import numpy as np
import pyworld
import soundfile as sf
from numpy.linalg import norm
from dtw import dtw
import glob
import pickle

# get wavefile info
def get_wave_info(file_name):
    sound, sr = librosa.load(file_name)
    mfcc = librosa.feature.mfcc(y = sound, sr = sr)
    stft = librosa.core.stft(sound)
    return sr, mfcc, stft

# do DTW
def DTW(mfcc_1, mfcc_2):
    dist, cost, acc_cost, path = dtw(mfcc_1.T, mfcc_2.T, dist = lambda x, y: norm(x - y, ord = 1))
    return path

# stft DTW
def DTW_stft(stft, path):
    dtw_stft = stft.T[path].T
    return dtw_stft

# stft to sound
def stft_to_sound(stft):
    conv_sound = librosa.core.istft(stft)
    return conv_sound

# data to wave
def make_sound(sound, sr, file_name):
    sound_double = sound.astype(np.float64)
    f0, sp, ap = pyworld.wav2world(sound_double, sr)
    make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
    sf.write(file_name, make_sound, sr)
    
def make_real_list(stft_list):
    real_list = []
    for stft in stft_list:
        real = np.empty_like(stft, dtype = float)
        for index in range(stft.shape[0]):
            for element in range(stft.shape[1]):
                real[index][element] = stft[index][element].real
        real_list.append(real)
    return real_list

def make_imag_list(stft_list):
    imag_list = []
    for stft in stft_list:
        imag = np.empty_like(stft, dtype = float)
        for index in range(stft.shape[0]):
            for element in range(stft.shape[1]):
                imag[index][element] = stft[index][element].imag
        imag_list.append(imag)
    return imag_list
    
# seika_info
seika_mfccs = []
seika_stfts = []
voices = glob.glob('../voice/seika_silence/*')
for voice in voices:
    sr, mfcc, stft = get_wave_info(voice)
    seika_mfccs.append(mfcc)
    seika_stfts.append(stft)

# hiro_info
hiro_mfccs = []
hiro_stfts = []
voices = glob.glob('../voice/hiro_silence/*')
for voice in voices:
    sr, mfcc, stft = get_wave_info(voice)
    hiro_mfccs.append(mfcc)
    hiro_stfts.append(stft)

# DTW
seika_conv_stfts = []
hiro_conv_stfts = []
for index in range(90):
    path = DTW(seika_mfccs[index], hiro_mfccs[index])
    seika_conv_stfts.append(DTW_stft(seika_stfts[index], path[0]).T)
    hiro_conv_stfts.append(DTW_stft(hiro_stfts[index], path[1]).T)
    

# save list
seika_stft = open('seika_stft.txt', 'wb')
pickle.dump(seika_conv_stfts, seika_stft)

hiro_stft = open('hiro_stft.txt', 'wb')
pickle.dump(hiro_conv_stfts, hiro_stft)

seika_real = open('seika_real.txt','wb')
pickle.dump(make_real_list(seika_conv_stfts), seika_real)

seika_imag = open('seika_imag.txt','wb')
pickle.dump(make_imag_list(seika_conv_stfts), seika_imag)

hiro_real = open('hiro_real.txt','wb')
pickle.dump(make_real_list(hiro_conv_stfts), hiro_real)

hiro_imag = open('hiro_imag.txt','wb')
pickle.dump(make_imag_list(hiro_conv_stfts), hiro_imag)

In [24]:
import librosa
import numpy as np
import pyworld
import soundfile as sf
#from numpy.linalg import norm
#from dtw import dtw
#import glob
import pickle

from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution1D
from keras.layers.advanced_activations import LeakyReLU

NUM_EPOCH = 20

# load list
f = open("./seika_stft.txt", "rb")
seika_stft = pickle.load(f)

f = open("./hiro_stft.txt", "rb")
hiro_stft = pickle.load(f)

f = open("./seika_real.txt", "rb")
seika_real = pickle.load(f)

f = open("./seika_imag.txt", "rb")
seika_imag = pickle.load(f)

f = open("./hiro_real.txt", "rb")
hiro_real = pickle.load(f)

f = open("./hiro_imag.txt", "rb")
hiro_imag = pickle.load(f)

def stft_to_sound(stft):
    conv_sound = librosa.core.istft(stft)
    return conv_sound

def make_sound(sound, sr, file_name):
    sound_double = sound.astype(np.float64)
    f0, sp, ap = pyworld.wav2world(sound_double, sr)
    make_sound = pyworld.synthesize(f0, sp, ap, sr, pyworld.default_frame_period)
    sf.write(file_name, make_sound, sr)

def conv_imag_list(real_list, imag_list):
    conv_list = []
    for index in range(len(real_list)):
        conv = np.empty_like(real_list[index], dtype = np.complex)
        for num in range(real_list[index].shape[0]):
            for element in range(real_list[index].shape[1]):
                conv[num][element] = complex(real_list[index][num][element], imag_list[index][num][element])
        conv_list.append(conv)
    return conv_list
        
def generator_model():
    model = Sequential()
    model.add(Dense(3075, input_dim = 1025))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dense(1025))
    model.add(Activation('tanh'))
    return model
    
def discriminator_model():
    model = Sequential()
    model.add(Dense(3075, input_dim = 1025))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model
    
def train_real():
    d_opt = Adam(lr = 1e-5, beta_1 = 0.1)
    discriminator = discriminator_model()
    discriminator.compile(loss = 'binary_crossentropy', optimizer = d_opt)
    
    discriminator.trainable = False
    generator = generator_model()
    dcgan = Sequential([generator, discriminator])
    g_opt = Adam(lr = 2e-4, beta_1 = 0.5)
    dcgan.compile(loss = 'binary_crossentropy', optimizer = g_opt)
    for epoch in range(NUM_EPOCH):
        i = 0
        for real in hiro_real:
            generated_real = generator.predict(real, verbose = 0)
        
            X = np.concatenate((seika_real[i], generated_real))
            y = [1] * real.shape[0] + [0] * real.shape[0]
            d_loss = discriminator.train_on_batch(X, y)
            g_loss = dcgan.train_on_batch(real, [1] * real.shape[0])

            i = i + 1
        
    return generated_real

def train_imag():
    d_opt = Adam(lr = 1e-5, beta_1 = 0.1)
    discriminator = discriminator_model()
    discriminator.compile(loss = 'binary_crossentropy', optimizer = d_opt)
    
    discriminator.trainable = False
    generator = generator_model()
    dcgan = Sequential([generator, discriminator])
    g_opt = Adam(lr = 2e-4, beta_1 = 0.5)
    dcgan.compile(loss = 'binary_crossentropy', optimizer = g_opt)
    for epoch in range(NUM_EPOCH):
        i = 0
        for imag in hiro_imag:
            generated_imag = generator.predict(imag, verbose = 0)
        
            X = np.concatenate((seika_imag[i], generated_imag))
            y = [1] * imag.shape[0] + [0] * imag.shape[0]
            d_loss = discriminator.train_on_batch(X, y)
            g_loss = dcgan.train_on_batch(imag, [1] * imag.shape[0])

            i = i + 1
        
    return generated_imag

gen_real = []
gen_imag = []
gen_real.append(train_real())
gen_imag.append(train_imag())
gen_sound = stft_to_sound(conv_imag_list(gen_real, gen_imag)[0].T)
make_sound(gen_sound, 22050, "make_sound.wav")

* batch
* conv